# This script builds Docker container 

In [ ]:
!pip install sagemaker-studio-image-build

In [ ]:
%%time
import os

import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

role = (
    get_execution_role()
)  # provide a pre-existing role ARN as an alternative to creating a new role
print(f"SageMaker Execution Role: {role}")

client = boto3.client("sts")
account = client.get_caller_identity()["Account"]
print(f"AWS account: {account}")

session = boto3.session.Session()
region = session.region_name
print(f"AWS region: {region}")

sm_boto_client = boto3.client("sagemaker")
sagemaker_session = sagemaker.session.Session(boto_session=session)

# get default bucket
default_bucket = sagemaker_session.default_bucket()
print("Default bucket for this session: ", default_bucket)

In [ ]:
!sm-docker build .

In [ ]:
!pwd

In [ ]:
%%writefile Dockerfile

# Set the default value for the REGION build argument
ARG REGION=us-west-2

# SageMaker PyTorch image for TRAINING
FROM 763104351884.dkr.ecr.${REGION}.amazonaws.com/pytorch-training:2.3.0-gpu-py311-cu121-ubuntu20.04-sagemaker

# Uninstall existing PyTorch packages
RUN pip uninstall torch torchvision transformer-engine -y

# Install latest release of PyTorch and torchvision
RUN pip install --force-reinstall torch==2.4.0

# Display installed packages for reference
RUN pip freeze

In [ ]:
!sm-docker build . --repository accelerate:latest